# AI Multimodal Gemology Assistant

### Overview
This notebook walks you through a gemology assistant that prices diamonds using real data:
- Load a diamonds pricing dataset from Kaggle.
- Define a tool that looks up prices (no manual guessing).
- Enforce required diamond attributes before calling the tool.
- Answer in a short, user-friendly way, with optional text-to-speech.
- Serve everything in a Gradio chat interface.



In [ ]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import kagglehub
import pandas as pd

In [ ]:
# !pip install kagglehub[pandas-datasets]
# download locally (to kagglehub cache)
path = kagglehub.dataset_download("nancyalaswad90/diamonds-prices")

In [ ]:
def get_diamods_df():
    # load the CSV (fix encoding issue)
    df = pd.read_csv(os.path.join(path, "Diamonds Prices2022.csv"), encoding="latin1")
    return df.copy()

In [ ]:
df = get_diamods_df()
df.sample(5)

In [ ]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [ ]:
system_message = """
You are a professional gemologist assistant.

If the user want to assess a diamond’s price, you MUST use the get_diamond_price tool.
You are NOT allowed to estimate or guess prices yourself.

Before calling the get_diamond_price tool, you MUST obtain ALL of the following parameters from the user:
- carat
- cut
- color
- clarity
- depth
- table
- x
- y
- z

If any required parameter is missing, you MUST ask the user for it and MUST NOT call the tool.
Only after all required parameters are provided may you call the tool and then explain the result.
Give short, courteous answers, no more than 2 sentences.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
def get_diamond_price(diamond):
    print(f"DATABASE TOOL CALLED: {diamond}", flush=True)

    filtered = get_diamods_df()
    
    for key, value in diamond.items():
        filtered = filtered[filtered[key] == value]

    if filtered.empty:
        return "No matching diamond found in the database"

    price = filtered["price"].median()
    return f"Estimated diamond price is ${price:.2f}"

In [ ]:
# Test
get_diamond_price({"carat": 0.23, "cut": "Ideal", "color": "E", "clarity": "SI2", "depth": 61.5, "table": 55, "x": 3.95, "y": 3.98, "z": 2.43})

In [ ]:
price_function = {
    "name": "get_diamond_price",
    "description": (
        "Get a diamond price from the database. "
        "This is the ONLY way to obtain a diamond price. "
        "Do NOT estimate prices without calling this tool."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "carat": {"type": "number"},
            "cut": {"type": "string"},
            "color": {"type": "string"},
            "clarity": {"type": "string"},
            "depth": {"type": "number"},
            "table": {"type": "number"},
            "x": {"type": "number"},
            "y": {"type": "number"},
            "z": {"type": "number"}
        },
        "required": ["carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_diamond_price":
            print("TOOL HANDLER: get_diamond_price", flush=True)
            diamond_info = json.loads(tool_call.function.arguments)
            price_details = get_diamond_price(diamond_info)

            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="coral",
      input=message
    )
    return response.content

In [ ]:
def chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    voice = talker(reply)
    return history, voice

In [ ]:
def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

# UI definition

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=300, type="messages")
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our Gemology AI Assistant:")

# Hooking up events to callbacks

    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, audio_output]
    )

ui.launch()